# Merge Election and Cabinet Data
This notebook merges each election with the next cabinet formed after the election, including all cabinet variables.

In [ ]:
import pandas as pd

# Load Excel files
cabinet_file = "Cabinet_data.xlsx"
election_file = "election_data_weights.xlsx"

cabinet_df = pd.read_excel(cabinet_file)
election_df = pd.read_excel(election_file)

# Convert date columns to datetime
cabinet_df["gov_start_date"] = pd.to_datetime(cabinet_df["gov_start_date"])
election_df["election_date"] = pd.to_datetime(election_df["election_date"])

# Sort for consistency
cabinet_df.sort_values(by=["countryID", "gov_start_date"], inplace=True)
election_df.sort_values(by=["countryID", "election_date"], inplace=True)

In [ ]:
# Merge logic
merged_rows = []

for (country, election_date), group in election_df.groupby(["countryID", "election_date"]):
    cabinets_country = cabinet_df[cabinet_df["countryID"] == country]
    next_cabinet_date = cabinets_country[cabinets_country["gov_start_date"] > election_date]["gov_start_date"].min()
    
    if pd.isna(next_cabinet_date):
        continue

    matching_cabinet = cabinets_country[cabinets_country["gov_start_date"] == next_cabinet_date]

    merged = pd.merge(
        group,
        matching_cabinet,
        on="partyID",
        how="left",
        suffixes=("", "_cabinet")
    )

    merged_rows.append(merged)

final_df = pd.concat(merged_rows, ignore_index=True)

In [ ]:
# Save to Excel
final_df.to_excel("full_merged_election_cabinet_data.xlsx", index=False)
print("✅ Merging complete! Output saved as 'full_merged_election_cabinet_data.xlsx'")